## Guide to setting up Pi-hole DNS server (using MacOS)

Equipment
 - Raspberry Pi Zero WH
 - Micro SDHC 16 GB
 - Power supply ?

### Install 

1. Download OS image

    a. Raspberry Pi OS Lite https://www.raspberrypi.com/software/operating-systems/

    b. DietPi https://dietpi.com (another option)

2. Format SD Card

    a. Run 'Disk Utility'

    b. Format as FAT32

3. Write OS image to SD card

    a. Launch 'Terminal'
    
    b. View list of drives: 'diskutil list'
    c. Unmount SD card: 'diskutil unmountDisk /dev/disk4' (replace disk4 with appropriate disk#)
    d. Write raspberry Pi OS image: 'sudo dd bs=1m if=<name.img> of=<disk>
    'sudo dd bs=1m if=Downloads/raspberrypioslite.img of=/dev/disk4'
        - name.img is location and name of downloaded OS image ex: 'Downloads/raspberrypioslite.img'
        - disk is appropriate SD disk. ex: /dev/disk4/
    e. Enter password when prompted
        - if 'not permitted', need to enable terminal disk utility privlidges
            System Preferences > Security & Privacy > Full Disk Access > Unlock > select Terminal, then restart terminal

4. Enable ssh and wifi connect files on 'boot'
    a. create ssh file: 'touch /Volumes/boot/ssh'
    b. create file to trigger avahi-daemon (allows usb tethering for ssh): 'touch /Volumes/boot/avahi' 
    b. edit 'config.txt'
        'cd /Volumes/boot'
        'open -a "TextEdit" config.txt'
        - append this line to the bottom of it disk
        'dtoverlay=dwc2'
        - save the file
    c. edit 'cmdline.txt'
        'cd /Volumes/boot'
        'open -a "TextEdit" cmdline.txt'
        - insert 'modules-load=dwc2,g_ether' after "rootwait" (make sure to leave only one space after rootwait and one space after the inserted text)
        - save the file
    d. create network info file: 'touch /Volumes/boot/wpa_supplicant.conf'
    e. add network info to file
        'cd /Volumes/boot'
        'open -a "TextEdit" wpa_supplicant.conf'
    f. copy info to wpa_supplicant.conf (adjust country code, ssid, psk)

country=US
ctrl_interface=DIR=/var/run/wpa_supplicant GROUP=netdev
update_config=1

network={
    ssid="NETWORK-NAME"
    psk="NETWORK-PASSWORD"
}

5. Eject SD card
    'diskutil eject /dev/disk4'

6. Boot raspberry pi
    a. Input SD card to raspberry pi
    b. Plug in usb micro cable to usb port on raspberry pi (not power)
    c. Plug in usb cable to computer
    d. Allow Raspberry pi to boot up (~90 seconds)
    e. Check connection
        Settings > Network > ?

7. Login over USB
    'ssh-keygen -R raspberrypi.local'
    'ssh pi@raspberrypi.local'

    default hostname: pi
    default password: raspberry

    - Should see SSH enabled...
        pi@raspberrypi:~ $ 

8. Change hostname and password
    a. Enter pi configuration: 'sudo raspi-config'
    b. Select '1 System Options'
    c. Select 'S3 Password'
    d. Input new password: peterpan89
    e. Select 'S4 Hostname'
    f. Input new hostname: mypi-hole
    g. Expand file system
        Select 'Advanced options'
        Select 'Expand file system'
    f. Select 'Finish' at bottom
    g. Select 'Yes' when asked to reboot

9. Update raspberry pi
    a. Reestablish ssh connection
        'ssh pi@mypi.local'
    b. Update and upgrade (these take a while)
        'sudo apt-get update -y'
        'sudo apt-get upgrade -y'
        'sudo apt-get dist-upgrade -y'
    c. Disable avahi
        'systemctl disable avahi-daemon' (this failed?)

10. Keep raspberry pi from going to sleep
    a. 'iwconfig' (to see power management setting)
    b. 'sudo /sbin/iw wlan0 set power_save off' (turns off power management)
    c. Make sure it runs everytime on reboot
        'sudo nano /etc/rc.local'
        - input 'sudo /sbin/iw wlan0 set power_save off' above exit0 line (scroll down)
        - save file and exit: ^X, Y, Enter

11. Assign raspberry pi a static IP address
    - Google home app: Wifi > Setting cog > Advanced Networking > DHCP IP Resrevations > Add > pick number and save
    - 192.168.86.69

12. Install pi-hole
    a. ssh pi@mypi-hole.local
    b. curl -sSL https://install.pi-hole.net | bash
    c. Follow prompts:
        Input static IPv4 when prompted
        Confirm gateway 192.168.86.1
        Confirm settings are correct
        Admin install interface: yes
        Install web server: yes
        Log queries: yes
        Privacy mode: Show everything
        
    d. Settings: ...
    e. Save web interface password: rVncCBM0
    f. Check setup at web interface
        - http://192.168.86.69/admin/



13. Configure router
    a. Google home app: Wifi > Setting cog > Advanced Networking > LAN Settings > DHCP Pool
        - Set both Start and End to : 192.168.86.69

    b. Go back to pi-hole web interface dashboard
    c. Under Settings > DHCP
        - Select "DHCP server enabled"
        - Under "Range of IP addresses to handout", select a suitable range (.70 - .251)
        - Save changes
    d. Restart Google router Wifi > Setting cog > restart network
    e. http://pi.hole/admin

